# DKC1 2CIMPL EBER2 Extraction
> This pipeline is designed to extract and analyze regions of the human genome identified by [2CIMPL](https://pubmed.ncbi.nlm.nih.gov/32610124/) as interacting with the EBV-encoded RNA 2 ([EBER2](https://pubmed.ncbi.nlm.nih.gov/26951683/)) region of the Epstein-Barr Virus genome.

### Software Dependencies:
* Python3
* Perl (used for CTK/PCR Collapsing)
* [CTK](https://github.com/chaolinzhanglab/ctk) - PCR Collapsing
* FastQC
* SAMtools
* BEDtools
* Bowtie2

### Python Packages:
* BioPython
* PySAM
* datetime

### Import Packages:

In [ ]:
from datetime import datetime
import pysam
from Bio import SeqIO

### Parameters:
* genome_files - [bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml#the-bowtie2-build-indexer) genome files
* software_dir - CTK directory (fastq2collapse.pl)
* in_file - input FASTQ
* output_dir - default is current directory (all necessary output directories are generated on execution)
* bt2 threads - default is 16 ___(CHANGE THIS)___

In [2]:
## Update these as necessary
genome_files = ('path/to/genome/genome_prefix')
software_dir = ('.../path/to/ctk')
in_file      = ('.../path/to/fastq')
output_dir   = ('.')

## Double-check the number of threads available, the bt2 parameters in this notebook are tailored for cluster computing!
bt2_threads  = 16

SyntaxError: invalid syntax (2971181648.py, line 4)

### FastQC:

In [ ]:
!mkdir $output_dir/fastqc

In [ ]:
!fastqc -o $output_dir/fastqc --no-extract $in_file 

### PCR Collapse:
> This Perl script is provided through the CLIP Toolkt (CTK) and is used to collapse PCR duplicates and greatly reduce the size of input files. This step is crucial to the efficient performance of this pipeline, and can reduce the original sanple size by around 60% in some applications.

In [ ]:
# -v makes it verbose
!perl $software_dir/fastq2collapse.pl -v $in_file $output_dir/collapsed.fastq

### Align with Bowtie2:
> The parameters used in this alignment are identical to those used in [Aligater's](https://github.com/timbitz/Aligater) native "align" function, which makes use of bowtie2 in a SAMtools-friendly wrapper. We used these parameters for the sake of continuity, as we had run previous analyses using aligater as well as much more relaxed bt2 parameters, and these parameters generated the highest number of identified EBER2 hybrids.

In [ ]:
!mkdir $output_dir/bowtie2

In [ ]:
## Feel free to play around with the quality threshold and k value, but when we ran this analysis with -k = 100 there were no changes
!bowtie2 -p $bt2_threads -k 50 -R 3 -N 0 -L 16 -i S,1,0.50 --local --reorder -x $genome_files \
-U $output_dir/collapsed.fastq 1> $output_dir/bowtie2/aligned.sam 2> $output_dir/bowtie2/bt2_align.log

#### Check BT2 output:
* "cat" the log file
* Should give the alignment rate, number of reads, and percentage of multi-mapped reads, as well as any errors thrown.
    * The percentage of multi-mapped reads should be much higher than single-mapped, since 2CIMPL generates hybrids through UV-crosslinking

In [ ]:
!cat $output_dir/bowtie2/bt2_align.log

## SAM Extraction & Analysis:

In [ ]:
!mkdir $output_dir/extract

In [ ]:
## Convert to BAM and sort
!samtools view -bS $output_dir/bowtie2/aligned.sam | samtools sort > $output_dir/extract/aligned.bam
## Index the BAM output
!samtools index $output_dir/extract/aligned.bam

### Extracting EBER2-specific reads:
* Use the samtools view -b function to extract the reads that mapped to chrEBV (or your chromosome of interest)
* Reads within a given range on said chromosome will have their IDs extracted & written to a text file, done by "awk-ing" the 4th column (start-of-alignment field for SAM files)

In [ ]:
## Extract the reads mapping to chrEBV, and select those reads that fall within our specified range
!samtools view -b $output_dir/extract/aligned.bam chrEBV | samtools view -h \
| awk '$4 >= 6961 && $4 <= 7135'> $output_dir/extract/EBER2_extracted.sam

In [ ]:
## Add a SAM header (-H) to the hybrid SAM file
!samtools view -H $output_dir/bowtie2/aligned.sam > $output_dir/bowtie2/aligned_header.sam
!cat $output_dir/bowtie2/aligned_header.sam $output_dir/extract/EBER2_extracted.sam > $output_dir/extract/EBER2_headed.sam

### Extracting EBER2-associated read IDs:
> In order to search the SAM file for hybrids, we first must extract the IDs. The theory is this: when 2CIMPL generates hybrid reads through UV crosslinking, a portion of each of the initial sequences will be maintained and subsequently align to their respective portions of the genome. With the knowledge that we are looking for EBER2-associated hybrids, if we identify all hybrids that mapped to EBER2 and some other portion of the genome, we can limit the field of search substantially. So, we just search the bowtie2 output file for our selected IDs and generate a new SAM file comprised of all EBER2 hybrid reads

In [ ]:
# Cut the first field (Read ID) and write the unqiue reads (no duplicates) to a text file
!samtools view -h $output_dir/extract/EBER2_headed.sam | cut -f 1 | sort | uniq > $output_dir/extract/EBER2_ids.txt
# Count the number of extracted IDs:
!wc -l $output_dir/extract/EBER2_ids.txt

## Extracting Hybrids from SAM and FASTQ files:
> This code is fairly optimal for searching decently large FASTQ and SAM files, but for very large files it is best to run the following chunks on a computing cluster where available. We were able to do this by initiating a Conda environment and importing the necessary methods.

### Extracting from FASTQ:
> Check each record in the FASTQ, see if it's in our set of wanted IDs. If it is, write to an output FASTQ. This is a handy way to cross-reference the mapping quality and sequences, especially when identifying which section of the hybrid is associated to the aligned region.

In [ ]:
ids = output_dir+"/extract/EBER2_ids.txt"

In [ ]:
def fastq_extract(fastq_in, fastq_out, ids):
    
    # Generate a set of IDs to facillitate faster searching as compared to a list
    wanted_ids = set(x[:-1] for x in open(ids))
    
    fastq_start = datetime.now()
    
    output_handle = open(fastq_out, "w")

    wanted = (record for record in SeqIO.parse(in_file,"fastq") if record.id in wanted_ids)
    SeqIO.write(wanted, output_handle, "fastq")
    output_handle.close()

    fastq_end = datetime.now()-fastq_start
    print("total FASTQ extraction time: ", fastq_end.seconds/60, " minutes")

In [ ]:
## Extract reads from initial FASTQ file
fastq_in  = in_file
fastq_out = output_dir+"/extract/EBER2_hybrids.fastq"

fastq_extract(fastq_in, fastq_out, ids)

## Count number of reads in output FASTQ file
!awk '{s++}END{print s/4}' $fastq_out

### Extracting from SAM:
> Input is a BAM file, output is a SAM file. PySam's fetch iterator is very quick, which is why it was chosen for this application. The read IDs are read into a map rather than a set in this application, which performed better when coupled with PySam fetch. 

In [ ]:
def sam_extract(ibam, osam, ids):
    sam_start = datetime.now()

    qnames = set(map(str.strip, open(ids)))
    bam_in = pysam.AlignmentFile(ibam)
    sam_out = pysam.AlignmentFile(osam, "w", template=bam)

    for b in bam_in.fetch(until_eof=True):
        if b.query_name in qnames:
            sam_out.write(b)
    bam_in.close()
    sam_out.close()

    sam_end = datetime.now()-sam_start

    print("total SAM extraction time: ",sam_end.seconds/60," minutes" )

In [ ]:
ibam = output_dir+"/extract/aligned.bam"
osam = output_dir+"/extract/EBER2_hybrids.sam"

sam_extract(ibam, osam, ids)

## Count number of reads in SAM file
count = !wc -l $osam
int((str(count).split())[1])-30

### Generate the Bedgraph:
> For post-processing, it's best to open the bedgraph in Excel and format it as a table. The result is a more permanent record that's human readable and can be readily sorted and re-sorted, which is why we avoided generating  and exporting a dataframe within Jupyter. For this analysis, we added an additional column to the Excel sheet documenting the mapping postion in the human genome and all associated genes/snoRNA identified through UCSC GB.

In [ ]:
!samtools view -bS $output_dir/extract/EBER2_hybrids.sam > $output_dir/extract/EBER2_hybrids.bam

In [ ]:
!bedtools genomecov -ibam $output_dir/extract/EBER2_hybrids.bam -bg | awk '$1 != "chrEBV"' > $output_dir/extract/EBER2_hybrids.bedgraph